In [1]:
%matplotlib notebook

In [132]:
import numpy as np
import matplotlib.pyplot as plt
from soprano.nmr.utils import _dip_tensor

$$
R_u(\theta)\mathbf{r} = \mathbf{u}(\mathbf{u}\cdot\mathbf{r}) + \cos(\theta)(\mathbf{u}\times\mathbf{r})\times\mathbf{u} + \sin(\theta)(\mathbf{u}\times\mathbf{r})
$$

$$
\begin{split}
\langle (R_u(\theta)\mathbf{r})\otimes(R_u(\theta)\mathbf{r}) \rangle = &(\mathbf{u}\otimes\mathbf{u})(\mathbf{u}\cdot\mathbf{r})^2 + \frac{1}{2}[(\mathbf{u}\times\mathbf{r})\times\mathbf{u}]\otimes[(\mathbf{u}\times\mathbf{r})\times\mathbf{u}] + \frac{1}{2}(\mathbf{u}\times\mathbf{r})\otimes (\mathbf{u}\times\mathbf{r}) = \\
&(\mathbf{u}\otimes\mathbf{u})(\mathbf{u}\cdot\mathbf{r})^2 + \frac{1}{2}[\mathbf{r}-(\mathbf{u}\cdot\mathbf{r})\mathbf{u}]\otimes[\mathbf{r}-(\mathbf{u}\cdot\mathbf{r})\mathbf{u}] + \frac{1}{2}(\mathbf{u}\times\mathbf{r})\otimes (\mathbf{u}\times\mathbf{r}) = \\
&\frac{3}{2}(\mathbf{u}\otimes\mathbf{u})(\mathbf{u}\cdot\mathbf{r})^2 + \frac{1}{2}\mathbf{r}\otimes\mathbf{r}-\frac{1}{2}(\mathbf{u}\cdot\mathbf{r})(\mathbf{r}\otimes\mathbf{u}+\mathbf{u}\otimes\mathbf{r})
 + \frac{1}{2}(\mathbf{u}\times\mathbf{r})\otimes (\mathbf{u}\times\mathbf{r}) = 
\end{split}
$$

In [163]:
0.5*(3*ur**2-1)*makeouter(u)+0.5*(1-ur**2)*np.eye(3)

array([[ 0.45 ,  0.   ,  0.   ],
       [ 0.   ,  0.275, -0.175],
       [ 0.   , -0.175,  0.275]])

In [189]:
r_par = (r@u)*u
r_ortho = r-r_par
np.linalg.eigh(makeouter(r_ortho))

(array([-3.21115865e-19,  1.41989037e-17,  1.09090909e-01]),
 array([[-0.98136037, -0.14758152,  0.12309149],
        [-0.06872714,  0.86767064,  0.49236596],
        [-0.17946699,  0.47472872, -0.86164044]]))

In [191]:
np.linalg.eigh(np.eye(3)-makeouter(u))

(array([-1.94289029e-16,  1.00000000e+00,  1.00000000e+00]),
 array([[ 0.90453403, -0.34896553, -0.24503315],
        [ 0.30151134,  0.92978979, -0.21114414],
        [ 0.30151134,  0.11710679,  0.94624358]]))

In [200]:
r@(makeouter(r)-(u@r)**2*makeouter(u))@r

0.011900826446280972

In [216]:
np.average([makeouter(R@r_ortho, R@r_ortho) for R in Rs], axis=0)

array([[ 0.00991736, -0.01487603, -0.01487603],
       [-0.01487603,  0.04958678, -0.00495868],
       [-0.01487603, -0.00495868,  0.04958678]])

In [220]:
(r_ortho@r_ortho)*(np.eye(3)-makeouter(u))/2

array([[ 0.00991736, -0.01487603, -0.01487603],
       [-0.01487603,  0.04958678, -0.00495868],
       [-0.01487603, -0.00495868,  0.04958678]])

In [190]:
makeouter(r_ortho)+makeouter(r_ortho, r_par)+makeouter(r_par, r_ortho)

array([[ 0.07107438,  0.15702479, -0.24297521],
       [ 0.15702479,  0.11900826, -0.08099174],
       [-0.24297521, -0.08099174, -0.08099174]])

In [182]:
np.linalg.norm(r_ortho)**2

0.10909090909090909

In [210]:
def makecross(u):
    return np.array([[0, -u[2], u[1]], [u[2], 0, -u[0]], [-u[1], u[0], 0]])

def makeouter(a, b=None):
    if b is None:
        b = a
    return a[:,None]*b[None,:]

def makerot(u, theta=0):
    u = np.array(u)/np.linalg.norm(u)
    ct = np.cos(theta)
    st = np.sin(theta)
    
    R = ct*np.eye(3) + st*makecross(u) + (1-ct)*makeouter(u)
    
    return R

def makeavgouter(r, u):
    return (0.5*makeouter(r) -
            0.5*makecross(u)@makeouter(r)@makecross(u)+
            1.5*makeouter(u)*(u@r)**2-
            0.5*(u@r)*(makeouter(u,r)+makeouter(r,u)))

In [211]:
u = np.array([3,1,1.0])
u /= np.linalg.norm(u)
r = np.array([1, 0.5, 0])
r /= np.linalg.norm(r)

rr = makeouter(r)
dd = _dip_tensor(1,r)

In [212]:
# Numerical average
thetas = np.linspace(0, 2*np.pi, 101)[:-1]
Rs = [makerot(u, th) for th in thetas]
dds = [R.T@dd@R for R in Rs]
rs = [R@r for R in Rs]
rrs = [R@rr@R.T for R in Rs]

print(np.average(dds, axis=0))
print(_dip_tensor(1.0, r, u))

print()

print(np.average(rrs, axis=0))
print(makeavgouter(r, u))
ur = u@r
print(0.5*(3*ur**2-1)*makeouter(u)+0.5*(1-ur**2)*np.eye(3))

[[ 1.21652893  0.68429752  0.68429752]
 [ 0.68429752 -0.60826446  0.22809917]
 [ 0.68429752  0.22809917 -0.60826446]]
[[ 1.21652893  0.68429752  0.68429752]
 [ 0.68429752 -0.60826446  0.22809917]
 [ 0.68429752  0.22809917 -0.60826446]]

[[0.73884298 0.22809917 0.22809917]
 [0.22809917 0.13057851 0.07603306]
 [0.22809917 0.07603306 0.13057851]]
[[0.73884298 0.22809917 0.22809917]
 [0.22809917 0.13057851 0.07603306]
 [0.22809917 0.07603306 0.13057851]]
[[0.73884298 0.22809917 0.22809917]
 [0.22809917 0.13057851 0.07603306]
 [0.22809917 0.07603306 0.13057851]]


In [136]:
# Analytical
# vp2 = np.dot(r, a)**2
# D = 0.5*d*(3*vp2-1)*(3*a[:, None]*a[None, :]-np.eye(3))

ur = u@r

0.5*(3*ur**2-1)*makeouter(u)+0.5*(1-ur**2)*np.eye(3)

array([[ 0.45 ,  0.   ,  0.   ],
       [ 0.   ,  0.275, -0.175],
       [ 0.   , -0.175,  0.275]])

In [137]:
makeouter(np.cross(u,r))

array([[ 0.1, -0.2,  0.2],
       [-0.2,  0.4, -0.4],
       [ 0.2, -0.4,  0.4]])

In [138]:
makeouter(u,r)+makeouter(r,u)

array([[0.        , 0.63245553, 0.63245553],
       [0.63245553, 0.63245553, 0.31622777],
       [0.63245553, 0.31622777, 0.        ]])

In [142]:
makeouter(r)@makecross(u)

array([[ 0.28284271, -0.56568542,  0.56568542],
       [ 0.14142136, -0.28284271,  0.28284271],
       [ 0.        ,  0.        ,  0.        ]])

In [143]:
makecross(u)@makeouter(r)

array([[-0.28284271, -0.14142136,  0.        ],
       [ 0.56568542,  0.28284271,  0.        ],
       [-0.56568542, -0.28284271,  0.        ]])

In [144]:
makecross(u)@makecross(u)

array([[-1. ,  0. ,  0. ],
       [ 0. , -0.5,  0.5],
       [ 0. ,  0.5, -0.5]])

In [145]:
makeouter(u)-np.eye(3)

array([[-1. ,  0. ,  0. ],
       [ 0. , -0.5,  0.5],
       [ 0. ,  0.5, -0.5]])

In [146]:
makecross(u)@makeouter(r)@makecross(u).T

array([[ 0.1, -0.2,  0.2],
       [-0.2,  0.4, -0.4],
       [ 0.2, -0.4,  0.4]])